## imports and files

In [22]:
import pandas as pd
import numpy as np
import os
import json

In [7]:
config_key_mapper = "key_config_mapper.csv"
results_addr = "."

## config map reader

In [4]:
def config_reader(csv_addr):
    key_config_df = pd.read_csv(csv_addr)
    return key_config_df
    

In [8]:
key_config_df = config_reader(config_key_mapper)
key_config_df

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,replicas
0,0,paper-audio-qa,audio,facebook/s2t-small-librispeech-asr,4,4Gi,5,1,1,10,1
1,2,paper-audio-qa,audio,facebook/s2t-small-librispeech-asr,4,4Gi,5,1,1,10,1
2,3,paper-audio-qa,audio,facebook/s2t-small-librispeech-asr,4,4Gi,5,1,1,10,1


In [20]:
def get_result_file_names(results_addr):
    files = []
    key_config = config_reader(config_key_mapper)
    experiments_keys = list(key_config["experiment_id"])
    for path in os.listdir(results_addr):
        if os.path.isfile(os.path.join(results_addr, path)):
            file_name = path.split(".")[0]
            try:
                if int(file_name) in experiments_keys:
                    files.append(path)
            except:
                pass
    return files

In [21]:
get_result_file_names(results_addr=results_addr)

['3.json']

## Read resutls

In [23]:
def read_results(results_addr, return_type = "all", specific_file = None ):
    files = get_result_file_names(results_addr=results_addr)
    
    if return_type == "all":
        return_results = {}
        for f in files:
            name = f.split(".")[0].split("/")[-1]
            json_file = open(f)
            return_results[name] = json.load(json_file)
        return return_results

            
read_results(results_addr=results_addr)            
        

{'3': {'cpu_usage': [6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.600069775,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.731229521,
   6813.73122

In [60]:
def latency_calculator(results):
    latencies = []
    for result in results:
        data = result['outputs'][0]['data']
        model_name = result['model_name']
        arrival_key = "arrival_" + model_name
        serve_key   = "serving_" + model_name
        data = json.loads(data[0])
        time = data["time"]
        latency = time[serve_key] - time[arrival_key]
        latencies.append(latency)
    return sum(latencies) / len(latencies)
        

In [67]:
def result_processing(kind_processing = "all", metrics = ["all"], functions = [], specific_experiments = []):
    results = read_results(results_addr=results_addr)            
    final_dataframe = []
    if kind_processing == "all":   
        if metrics[0] == "all":
            for key in results.keys():
                current_result_key = [key]
                data = results[key]
                if len(functions) == 0:
                    for data_key in data.keys():
                        metric_result = data[data_key]
                        print(data_key)
                        if "time" in data_key:
                            current_result_key.append(metric_result[0])
                        elif data_key == "responses":
                            current_result_key.append(latency_calculator(metric_result))
                        else:
                            try:
                                current_result_key.append(max(metric_result))
                            except:
                                current_result_key.append(0)
                final_dataframe.append(current_result_key)
                
            
            return final_dataframe
        
        
        
        
        

In [70]:
results = result_processing()

cpu_usage
time_cpu
memory_usage
time_memory
responses
latency
throughput


In [69]:
headers = ["experiment_id", "cpu_usage", "time_cpu", "memory_usage", "time_memory", "latency_responses", "latency_from_seldon", "throughput"]

In [75]:

df = pd.DataFrame(results)


,0,1,2,3,4,5,6,7
0,3,6945.566498,1.666053e+09,1.035059e+09,1.666053e+09,5.438508,0,0
